In [1]:
import pandas as pd
import pickle
pd.set_option("display.max_columns", None)
from collections import Counter
import spacy
import google_conf

In [2]:
mops_data = google_conf.setup(sheet_url="https://docs.google.com/spreadsheets/d/1VbCIAJssHKV9hlRTwzVFfm40CGnHesq53KXjv2qy4OM/edit?usp=sharing", service_account_path="../../ServiceAccountsKey.json")

In [3]:
metadata_df = pd.read_json("../data/metadata_df.json")

In [4]:
metadata_df.head(5)

,creator,datePublished,docType,doi,id,identifier,isPartOf,issueNumber,keyphrase,language,outputFormat,pageCount,pageEnd,pageStart,pagination,provider,publicationYear,publisher,sequence,tdmCategory,title,url,volumeNumber,wordCount,id_kase,docSubType,sourceCategory,abstract,subTitle
0,[Gerald F. Moede],1974-04-01,article,10.1111/j.1758-6623.1974.tb02427.x,ark://27927/phx7f872pd,"[{'name': 'doi', 'value': '10.1111/j.1758-6623...",The Ecumenical Review,2,"[church, church union, christian unity, confes...",[eng],"[unigram, bigram, trigram]",19,264,246,246-264,portico,1974,"John Wiley & Sons, Inc.",7.0,"[Philosophy - Applied philosophy, Political sc...",CHURCH UNION AS A MODEL OF CHRISTIAN UNITY,http://doi.org/10.1111/j.1758-6623.1974.tb02427.x,26,7827,0,None,None,None,None
1,[Ben C. Ollenburger],1987-10-01,article,10.1177/004057368704400307,ark://27927/phx6238nb98,"[{'name': 'doi', 'value': '10.1177/00405736870...",Theology Today,3,"[haarlem, miep gies, german, suffering, hollan...",[eng],"[unigram, bigram, trigram]",10,359,350,pp. 350-359,portico,1987,SAGE Publications,7.0,[Philosophy - Applied philosophy],Suffering and Hope: The Story Behind the Book,http://doi.org/10.1177/004057368704400307,44,5181,1,None,None,None,None
2,[Eric L. Johnson],1997-03-01,article,10.1177/009164719702500102,ark://27927/phz2f6c9d3v,"[{'name': 'doi', 'value': '10.1177/00916471970...",Journal of Psychology and Theology,1,"[kingdom, christian, creation, lordship, knowl...",[eng],"[unigram, bigram, trigram]",17,27,11,pp. 11-27,portico,1997,SAGE Publications,2.0,[Philosophy - Metaphysics],"Christ, the Lord of Psychology",http://doi.org/10.1177/009164719702500102,25,13697,2,None,None,None,None
3,"[Kimberly Matheson, Hymie Anisman, Renate Ysse...",2010-02-01,article,10.1177/1088868309349693,ark://27927/pgfvbkcxrg,"[{'name': 'doi', 'value': '10.1177/10888683093...",Personality and Social Psychology Review,1,"[religious, identity, religious identity, soci...",[eng],"[unigram, bigram, trigram]",12,71,60,60-71,portico,2010,SAGE Publications,6.0,"[Social sciences - Psychology, Social sciences...",Religiosity as Identity: Toward an Understandi...,http://doi.org/10.1177/1088868309349693,14,9915,3,None,None,None,None
4,[David J. Clark],1975-01-01,article,10.1177/026009357502600107,ark://27927/phx5181bbsv,"[{'name': 'doi', 'value': '10.1177/02600935750...",The Bible Translator (1950-2012),1,"[haenchen, overboard, barclay, apostles, went ...",[eng],"[unigram, bigram, trigram]",3,146,144,pp. 144-146,portico,1975,SAGE Publications,7.0,"[Linguistics - Grammar, Religion - Spiritual b...",What Went Overboard First?,http://doi.org/10.1177/026009357502600107,26,1652,4,None,None,None,None


In [6]:
len(metadata_df[metadata_df["doi"].notnull()])

13100

In [7]:
metadata_df[metadata_df["doi"].notnull()].groupby("isPartOf").size().sort_values(ascending=False)

isPartOf
Religious Studies Review                                1726
Interpretation: A Journal of Bible and Theology          868
Expository Times                                         720
Review & Expositor: An International Baptist Journal     717
The Heythrop Journal                                     686
                                                        ... 
Journal of Croatian Studies                                1
Journal of Contemporary History                            1
Journal of Consumer Marketing                              1
Journal of Conflict Resolution                             1
Ériu                                                       1
Length: 821, dtype: int64

In [8]:
metadata_df[metadata_df["doi"].notnull()].head(10)

,creator,datePublished,docType,doi,id,identifier,isPartOf,issueNumber,keyphrase,language,outputFormat,pageCount,pageEnd,pageStart,pagination,provider,publicationYear,publisher,sequence,tdmCategory,title,url,volumeNumber,wordCount,id_kase,docSubType,sourceCategory,abstract,subTitle
0,[Gerald F. Moede],1974-04-01,article,10.1111/j.1758-6623.1974.tb02427.x,ark://27927/phx7f872pd,"[{'name': 'doi', 'value': '10.1111/j.1758-6623...",The Ecumenical Review,2,"[church, church union, christian unity, confes...",[eng],"[unigram, bigram, trigram]",19,264,246,246-264,portico,1974,"John Wiley & Sons, Inc.",7.0,"[Philosophy - Applied philosophy, Political sc...",CHURCH UNION AS A MODEL OF CHRISTIAN UNITY,http://doi.org/10.1111/j.1758-6623.1974.tb02427.x,26,7827,0,None,None,None,None
1,[Ben C. Ollenburger],1987-10-01,article,10.1177/004057368704400307,ark://27927/phx6238nb98,"[{'name': 'doi', 'value': '10.1177/00405736870...",Theology Today,3,"[haarlem, miep gies, german, suffering, hollan...",[eng],"[unigram, bigram, trigram]",10,359,350,pp. 350-359,portico,1987,SAGE Publications,7.0,[Philosophy - Applied philosophy],Suffering and Hope: The Story Behind the Book,http://doi.org/10.1177/004057368704400307,44,5181,1,None,None,None,None
2,[Eric L. Johnson],1997-03-01,article,10.1177/009164719702500102,ark://27927/phz2f6c9d3v,"[{'name': 'doi', 'value': '10.1177/00916471970...",Journal of Psychology and Theology,1,"[kingdom, christian, creation, lordship, knowl...",[eng],"[unigram, bigram, trigram]",17,27,11,pp. 11-27,portico,1997,SAGE Publications,2.0,[Philosophy - Metaphysics],"Christ, the Lord of Psychology",http://doi.org/10.1177/009164719702500102,25,13697,2,None,None,None,None
3,"[Kimberly Matheson, Hymie Anisman, Renate Ysse...",2010-02-01,article,10.1177/1088868309349693,ark://27927/pgfvbkcxrg,"[{'name': 'doi', 'value': '10.1177/10888683093...",Personality and Social Psychology Review,1,"[religious, identity, religious identity, soci...",[eng],"[unigram, bigram, trigram]",12,71,60,60-71,portico,2010,SAGE Publications,6.0,"[Social sciences - Psychology, Social sciences...",Religiosity as Identity: Toward an Understandi...,http://doi.org/10.1177/1088868309349693,14,9915,3,None,None,None,None
4,[David J. Clark],1975-01-01,article,10.1177/026009357502600107,ark://27927/phx5181bbsv,"[{'name': 'doi', 'value': '10.1177/02600935750...",The Bible Translator (1950-2012),1,"[haenchen, overboard, barclay, apostles, went ...",[eng],"[unigram, bigram, trigram]",3,146,144,pp. 144-146,portico,1975,SAGE Publications,7.0,"[Linguistics - Grammar, Religion - Spiritual b...",What Went Overboard First?,http://doi.org/10.1177/026009357502600107,26,1652,4,None,None,None,None
5,[F. L. Shoemaker],1955-01-01,article,10.1111/j.1741-5446.1955.tb01287.x,ark://27927/phx29mqcsph,"[{'name': 'doi', 'value': '10.1111/j.1741-5446...",Educational Theory,1,"[idealism, universe, absolutes, metaphysical i...",[eng],"[unigram, bigram, trigram]",13,46,34,34-46,portico,1955,"John Wiley & Sons, Inc.",6.0,[Philosophy - Metaphysics],AN APPROACH TO METAPHYSICAL IDEALISTIC ECLECTI...,http://doi.org/10.1111/j.1741-5446.1955.tb01287.x,5,7605,5,None,None,None,None
6,[J. A. B. Holland],1961-09-01,article,10.1017/S0036930600003355,ark://27927/pgh1g2c67j2,"[{'name': 'local_publisher_id', 'value': 'S003...",Scottish Journal of Theology,3,"[irenaeus, adversus haereses, church, marcion,...",[eng],"[unigram, bigram, trigram]",6,317,312,312-317,portico,1961,Cambridge University Press,21.0,[History - Historical methodology],Review of Saint Irénée: Introduction à l'étude...,http://doi.org/10.1017/S0036930600003355,14,2848,6,None,None,None,None
7,[Jun Ki Chung],2010-07-01,article,10.1177/009182961003800302,ark://27927/phx5wv17ht2,"[{'name': 'doi', 'value': '10.1177/00918296100...",Missiology: An International Review,3,"[church, holy communion, korean, japanese, chr...",[eng],"[unigram, bigram, trigram]",17,269,253,pp. 253-269,portico,2010,SAGE Publications,2.0,[Religion - Spiritual belief systems],An

In [9]:
metadata_df["publicationYear"].mean()

1989.97008

# Journals...

In [10]:
journals_df = metadata_df.groupby("isPartOf").size().sort_values(ascending=False)
journals_df = journals_df.reset_index()
journals_df.rename(columns={0: "N"}, inplace=True)
journals_df.head(10)

,isPartOf,N
0,Religious Studies Review,1726
1,The Catholic Biblical Quarterly,893
2,Interpretation: A Journal of Bible and Theology,870
3,Journal of Biblical Literature,847
4,Expository Times,724
5,Review & Expositor: An International Baptist J...,719
6,The Heythrop Journal,686
7,The Journal of Theological Studies,492
8,New Testament Studies,412
9,Church History,379


In [11]:
journals_df_year_avr = metadata_df.groupby("isPartOf")["publicationYear"].mean().reset_index()
journals_df_year_avr.rename(columns={"publicationYear": "year_avr"}, inplace=True)

journals_df_year_min = metadata_df.groupby("isPartOf")["publicationYear"].min().reset_index()
journals_df_year_min.rename(columns={"publicationYear": "year_min"}, inplace=True)

journals_df_year_max = metadata_df.groupby("isPartOf")["publicationYear"].max().reset_index()
journals_df_year_max.rename(columns={"publicationYear": "year_max"}, inplace=True)

In [12]:
for df in [journals_df_year_avr, journals_df_year_min, journals_df_year_max]:
    journals_df = journals_df.merge(df, on="isPartOf")
journals_df.head(20)

,isPartOf,N,year_avr,year_min,year_max
0,Religious Studies Review,1726,2009.486674,1976,2021
1,The Catholic Biblical Quarterly,893,1991.070549,1939,2016
2,Interpretation: A Journal of Bible and Theology,870,1983.020690,1947,2021
3,Journal of Biblical Literature,847,1982.760331,1901,2021
4,Expository Times,724,1987.357735,1938,2022
5,Review & Expositor: An International Baptist J...,719,1981.000000,1932,2021
6,The Heythrop Journal,686,2012.384840,1960,2021
7,The Journal of Theological Studies,492,1978.888211,1900,2016
8,New Testament Studies,412,1988.890777,1954,2021
9,Church History,379,1986.543536,1934,2021


In [13]:
google_conf.set_with_dataframe(mops_data.add_worksheet("jstor_journals_minmaxavryear", 1, 1), journals_df.reset_index())

In [14]:
metadata_df.shape

(25000, 29)

In [15]:
metadata_df["wordCount"].sum()

232665684

In [17]:
metadata_df[mask_articles]["wordCount"].sum()

232665684

In [18]:
mask_doi = metadata_df["doi"].notnull()
len(metadata_df[mask_doi])

13100

In [20]:
mask_jstor = metadata_df["provider"]=="jstor"
sum((mask_articles) & (mask_jstor))

11205

In [18]:
unigramCount_dict = pickle.load(open("../data/large_files/unigramCount_dict.pickle", "rb"))

In [49]:
# manual test
col = "publicationYear"
matchstring = "==1951"
eval('metadata_df[metadata_df["{0}"]{1}]'.format(col, matchstring))["id"].tolist()

['http://www.jstor.org/stable/1198957',
 'http://www.jstor.org/stable/1508417',
 'ark://27927/phx7f9rxk2',
 'http://www.jstor.org/stable/3261590',
 'http://www.jstor.org/stable/24456097',
 'ark://27927/pgg389199fd',
 'ark://27927/phx61xr9tmg',
 'ark://27927/pgh1g2mt8q5',
 'ark://27927/phx61xrbpxv',
 'ark://27927/phx64dprn08',
 'http://www.jstor.org/stable/43720276',
 'http://www.jstor.org/stable/20516383',
 'http://www.jstor.org/stable/41175585',
 'ark://27927/phx61tbjnvh',
 'http://www.jstor.org/stable/1198933',
 'http://www.jstor.org/stable/1456587',
 'http://www.jstor.org/stable/43720275',
 'http://www.jstor.org/stable/44210186',
 'ark://27927/phx622zp8bt',
 'http://www.jstor.org/stable/1197811',
 'ark://27927/pgh1h1tfc4p',
 'http://www.jstor.org/stable/42621087',
 'ark://27927/phx61tbjqw3',
 'ark://27927/phx64dprn74',
 'ark://27927/pgh1h00bn1j',
 'ark://27927/pgh1g2mt8xg',
 'ark://27927/phx6bfsx4f4',
 'http://www.jstor.org/stable/1198956',
 'http://www.jstor.org/stable/3262039',
 '

In [19]:
def ids_from_colvals(col, matchstring):
    ids = eval('metadata_df[metadata_df["{0}"]{1}]'.format(col, matchstring))["id"].tolist()
    return ids

In [20]:
# test with function...
len(ids_from_colvals("publicationYear", ".between(1950, 1959)"))

403

In [21]:
test_ids = ids_from_colvals("publicationYear", "==1951")
len(test_ids)

37

In [22]:
unigramCount_dict[test_ids[0]]

{'NEIL.': 1,
 'assumption.': 1,
 'emphases': 1,
 'work': 2,
 "Neil's": 1,
 "man's": 1,
 'by-passed.': 1,
 'apply': 1,
 'others,': 1,
 'home.': 1,
 'or,': 1,
 'Acts.': 1,
 'pages': 1,
 'has': 2,
 'discus-': 1,
 'connection': 2,
 '"ten-': 1,
 'scanty': 1,
 'amid': 1,
 'treat-': 1,
 'mis-': 1,
 'Lord': 1,
 'II': 3,
 'on': 11,
 'One': 1,
 'writings.': 1,
 'indorse': 1,
 'dis-': 1,
 'text': 3,
 'intentions': 1,
 'Many': 1,
 'however,': 1,
 'insights': 1,
 'ac-': 1,
 'Sermon': 2,
 'of': 98,
 'xlv).': 1,
 'disagree-': 1,
 'Bros.,': 1,
 'Christians': 1,
 'authorities': 1,
 'eschatology.': 2,
 '79,': 1,
 '"restraining': 1,
 'charge': 1,
 'sion': 1,
 'will': 3,
 'so': 1,
 'modernizing': 1,
 'level': 1,
 'defined': 1,
 'blasphemous': 1,
 'epistles,': 1,
 '21,': 1,
 'York:': 2,
 'enemies': 2,
 'instance,': 1,
 'Introduction': 1,
 'reality': 2,
 'tion': 3,
 'treatment': 1,
 'adequate': 1,
 'indorses': 1,
 'Christianity.': 1,
 'data': 2,
 'interesting': 2,
 'series,': 1,
 'supposed': 1,
 'Account': 

In [64]:
c = Counter()
for id in test_ids:
    d = unigramCount_dict[id]
    print(len(d))
    c.update(d)

862
531
1250
1450
2219
1152
2181
1581
2134
2872
1247
1135
1255
1727
733
1218
4749
1875
1590
2344
2139
3184
2348
1404
2572
939
2297
799
2624
2775
1019
1914
1747
1284
1365
2629
519


In [69]:
def merge_data_from_ids(ids, datadict):
    c = Counter()
    for id in ids:
        d = datadict[id]
        c.update(d)
    return c

In [67]:
sorted(c.items(), key=lambda kv: kv[1], reverse=True)

[('the', 13802),
 ('of', 9030),
 ('and', 5024),
 ('to', 4495),
 ('in', 4398),
 ('is', 2709),
 ('a', 2576),
 ('that', 2244),
 ('as', 1425),
 ('The', 1367),
 ('for', 1293),
 ('his', 1261),
 ('which', 1239),
 ('be', 1180),
 ('was', 1162),
 ('not', 1148),
 ('he', 1101),
 ('by', 1057),
 ('it', 971),
 ('with', 968),
 ('this', 836),
 ('from', 818),
 ('on', 727),
 ('are', 726),
 ('have', 669),
 ('but', 648),
 ('.', 621),
 ('we', 600),
 ('who', 586),
 ('or', 539),
 ('had', 539),
 ('at', 538),
 ('has', 537),
 ('their', 497),
 ('an', 485),
 ('were', 479),
 ('one', 455),
 ('all', 446),
 ('they', 439),
 ('Paul', 409),
 ('God', 404),
 ('I', 396),
 ('its', 391),
 ('been', 390),
 ('In', 371),
 ('no', 368),
 ('It', 357),
 ('Testament', 339),
 ('will', 331),
 ('our', 331),
 ('Christian', 331),
 ('only', 313),
 ('would', 297),
 ('can', 282),
 ('New', 281),
 ('there', 279),
 ('more', 279),
 ('Old', 279),
 ('so', 277),
 ('into', 264),
 ('p.', 260),
 ('other', 258),
 ('must', 258),
 ('He', 256),
 ('Jesus', 

In [73]:
#english stopwords...
sp = spacy.load('en_core_web_sm')
stop_words = sp.Defaults.stop_words

In [74]:
type(stop_words)

set

In [109]:
def get_tops(col, matchstring, n=10):
    ids = ids_from_colvals(col, matchstring)
    c = merge_data_from_ids(ids, unigramCount_dict)
    c_tups = [el for el in c.items() if el[0].lower() not in list(stop_words) + [".", "-", "\""]]
    top10 = sorted(c_tups, key=lambda kv: kv[1], reverse=True)[:n]
    return top10

In [110]:
get_tops("publicationYear", "==1951", 20)

[('Paul', 409),
 ('God', 404),
 ('Testament', 339),
 ('Christian', 331),
 ('New', 281),
 ('Old', 279),
 ('p.', 260),
 ('Jesus', 248),
 ('man', 238),
 ('Christ', 237),
 ('Gospel', 233),
 ('life', 225),
 ('law', 208),
 ('love', 192),
 ('new', 173),
 ('Church', 170),
 ('St.', 168),
 (';', 167),
 ('men', 161),
 ("God's", 156)]

In [111]:
get_tops("publicationYear", ".between(1950,1959)")

[('God', 5002),
 ('Paul', 3773),
 ('Christian', 3543),
 ('Christ', 3098),
 ('New', 2981),
 ('Church', 2966),
 ('Jesus', 2862),
 ('p.', 2506),
 ('life', 2377),
 ('St.', 2364)]

In [112]:
for decade_n in range(0,10):
    print("19{0}0,19{0}9".format(str(decade_n)))

1900,1909
1910,1919
1920,1929
1930,1939
1940,1949
1950,1959
1960,1969
1970,1979
1980,1989
1990,1999


In [113]:
for decade_n in range(0,10):
    decade = "19{0}0,19{0}9".format(str(decade_n))
    print(decade, get_tops("publicationYear", ".between({0})".format(decade), 20))


1900,1909 [('Proc.', 3172), ('d.', 2347), (':', 2342), ('2', 2219), ('de', 2113), ('Jesus', 2013), ('u.', 1737), ('&', 1650), ('God', 1647), ('New', 1622), ('Christian', 1493), ('der', 1478), ('Paul', 1440), ('3', 1371), ('des', 1366), ('Testament', 1330), ('Christ', 1290), ('?', 1251), ('et', 1209), ('1', 1179)]
1910,1919 [('Jesus', 1776), ('God', 1736), ('Christian', 1542), ('great', 1405), ('church', 1200), ('life', 1182), ('time', 1177), ('p.', 1171), ('man', 1128), ('Paul', 1126), ('New', 1096), ('religious', 950), ('Christ', 905), ('men', 893), ('shall', 878), ('?', 858), (';', 845), ('found', 812), ('new', 794), ('work', 787)]
1920,1929 [('|', 7527), ('p.', 2363), ('pp.', 1682), ('New', 1568), (':', 1110), ('Christian', 1054), ('1', 900), ('work', 871), ('Paul', 852), ('St.', 845), (';', 811), ('great', 788), ('time', 764), ('God', 754), ('religious', 725), ('A.', 702), ('Jewish', 692), ('American', 683), ("'", 635), ('new', 629)]
1930,1939 [('et', 2512), ('p.', 1790), ('God', 1

In [102]:
bigramCount_dict = pickle.load(open("../data/large_files/bigramCount_dict.pickle", "rb"))

In [107]:
def get_top_bigrams(col, matchstring, n=10):
    ids = ids_from_colvals(col, matchstring)
    c = merge_data_from_ids(ids, bigramCount_dict)
    #c_tups = [el for el in c.items() if el[0].lower() not in list(stop_words)]
    tops = sorted(c.items(), key=lambda kv: kv[1], reverse=True)[:n]
    return tops

In [108]:
for decade_n in range(0,10):
    decade = "19{0}0,19{0}9".format(str(decade_n))
    print(decade, get_top_bigrams("publicationYear", ".between({0})".format(decade), 10))

1900,1909 [('of the', 21104), ('in the', 8596), ('to the', 5662), ('and the', 3227), ('- -', 3015), ('. .', 2882), ('that the', 2608), ('on the', 2244), ('by the', 2183), ('to be', 2122)]
1910,1919 [('of the', 18350), ('in the', 8101), ('to the', 5355), ('and the', 3300), ('that the', 2540), ('. .', 2416), ('on the', 2058), ('to be', 2054), ('from the', 2042), ('with the', 1988)]
1920,1929 [('of the', 13452), ('in the', 5457), ('. .', 4018), ('to the', 3686), ('and the', 2241), ('on the', 1742), ('for the', 1564), ('with the', 1452), ('from the', 1426), ('by the', 1419)]
1930,1939 [('of the', 13587), ('in the', 5959), ('to the', 4491), ('and the', 2441), ('that the', 1689), ('for the', 1599), ('by the', 1516), ('to be', 1460), ('with the', 1453), ('on the', 1420)]
1940,1949 [('of the', 21661), ('in the', 9715), ('to the', 6489), ('and the', 3567), ('that the', 3104), ('to be', 2730), ('on the', 2425), ('for the', 2410), ('with the', 2384), ('by the', 2219)]
1950,1959 [('of the', 37166)